# Audio Signal Classification Using Convolutional Neural Networks (CNNs)

In this notebook, we'll classify urban sounds using Convolutional Neural Networks (CNNs) with TensorFlow. We will use the UrbanSound8K dataset, convert audio signals into spectrograms, and train a CNN model to classify these spectrograms.

## 1. Setup

First, we need to install the necessary libraries and import them.

In [ ]:
!pip install tensorflow numpy librosa matplotlib
!pip install kaggle
# Make sure to upload kaggle.json to Colab if using Colab

In [ ]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Ensure the dataset is available
if not os.path.exists('UrbanSound8K.tar.gz'):
    !kaggle datasets download -d chrisdonahue/urbansound8k
    !tar -xzvf UrbanSound8K.tar.gz

# Define the path to the dataset
dataset_path = 'UrbanSound8K/audio/'
classes = os.listdir(dataset_path)
print('Classes:', classes)


## 2. Data Preparation

We need to load the audio files and convert them into spectrograms.

In [ ]:
def audio_to_spectrogram(file_path, n_mels=128, fmax=8000):
    y, sr = librosa.load(file_path, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)
    return librosa.power_to_db(S, ref=np.max)

# Collect data
X = []
y = []
for label, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    for file_name in os.listdir(class_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(class_path, file_name)
            spectrogram = audio_to_spectrogram(file_path)
            X.append(spectrogram)
            y.append(label)

X = np.array(X)
y = np.array(y)

# Normalize data
X = (X - np.min(X)) / (np.max(X) - np.min(X))
X = np.expand_dims(X, axis=-1)  # Add channel dimension
y = to_categorical(y, num_classes=len(classes))


## 3. Model Building

We'll build a CNN model to classify the spectrograms.

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 4. Training

We'll split the data into training and validation sets and then train our model.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)


## 5. Evaluation

Let's evaluate the model's performance on the validation set.

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy:.4f}')

# Plot training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()
